In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mglearn
import warnings

warnings.filterwarnings(action='ignore', module='scipy', message='^internal gelsd')
pd.options.display.max_rows = 15

%matplotlib inline

Зареждам данните (някои записи не са валидни, затова слагам error_bad_lines на false, което ще ги пропусне).

In [2]:
videos = pd.read_csv('data/USvideos.csv', error_bad_lines=False)
videos

b'Skipping line 2401: expected 11 fields, saw 21\nSkipping line 2800: expected 11 fields, saw 21\nSkipping line 5297: expected 11 fields, saw 12\nSkipping line 5299: expected 11 fields, saw 12\nSkipping line 5300: expected 11 fields, saw 12\nSkipping line 5301: expected 11 fields, saw 12\n'


,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09
5,cMKX2tE5Luk,The Disaster Artist | Official Trailer HD | A24,A24,1,a24|a24 films|a24 trailers|independent films|t...,1311445,34507,544,3040,https://i.ytimg.com/vi/cMKX2tE5Luk/default.jpg,13.09
6,8wNr-NQImFg,"The Check In: HUD, Ben Carson and Hurricanes",Late Night with Seth Meyers,23,Late night|Seth Meyers|check in|hud|Ben Carson...,666169,9985,297,1071,https://i.ytimg.com/vi/8wNr-NQImFg/default.jpg,13.09
...,...,...,...,...,...,...,...,...,...,...,...
7985,CN4YL1KPDAA,Pixel 2 Classic Unboxing,Linus Tech Tips,28,[none],484376,18242,797,1540,https://i.ytimg.com/vi/CN4YL1KPDAA/default.jpg,22.10
7986,4kmSATD25iY,Pumpkin Spice Oreo Taste Test!,iJustine,22,ijustine|pumpkin spice oreo|taste test|itsmeje...,164625,6688,383,1610,https://i.ytimg.com/vi/4kmSATD25iY/default.jpg,22.10


Да видим с какви данни разполагаме

In [3]:
videos.describe()

,category_id,views,likes,dislikes,comment_total,date
count,7992.000000,7.992000e+03,7.992000e+03,7992.000000,7992.000000,7992.000000
mean,20.217718,9.390269e+05,3.449193e+04,1781.625000,4608.489740,16.094249
std,7.415525,2.148263e+06,9.136160e+04,10822.103125,20252.758016,7.677674
min,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.100000
25%,17.000000,9.686375e+04,1.907500e+03,67.000000,286.000000,11.100000
50%,23.000000,3.086115e+05,8.652500e+03,272.500000,1085.500000,16.595000
75%,25.000000,9.588758e+05,3.021350e+04,1013.000000,3462.750000,21.100000
max,43.000000,4.150067e+07,2.010366e+06,318404.000000,736179.000000,30.090000


In [4]:
videos.isnull().sum().sort_values()

video_id          0
title             0
channel_title     0
category_id       0
tags              0
views             0
likes             0
dislikes          0
comment_total     0
thumbnail_link    0
date              0
dtype: int64

Всички данни са валидни, но някои не дават много информация. Махам ги за сега (video_id, thumbnail_link)

In [5]:
del videos['video_id']
del videos['thumbnail_link']

## Baseline Model

Ще искам да натренирам модел, който да предполага колко гледания има дадено видео.
Първо ще ползвам само числовите колони.

In [6]:
X = videos[['category_id', 'likes', 'dislikes', 'comment_total']]
y = videos['views']

print(X)
print(y)

      category_id   likes  dislikes  comment_total
0              24  320053      5931          46245
1              28  185853     26679              0
2              22  576597     39774         170708
3              28   24975      4542          12829
4              23   96666       568           6666
5               1   34507       544           3040
6              23    9985       297           1071
...           ...     ...       ...            ...
7985           28   18242       797           1540
7986           22    6688       383           1610
7987           27   14362       390           1575
7988           25    1265       760           1873
7989           10    7088        68            437
7990           24    2148        77              0
7991           28    1574        59              0

[7992 rows x 4 columns]
0       4394029
1       7860119
2       5845909
3       2642103
4       1168130
5       1311445
6        666169
         ...   
7985     484376
7986     164625

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, y_train)
print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

train score: 0.730928079438
test score:  0.651118174035


Изглежда ок, да пробваме обаче да обработим текстовите колони..

# Текстови колони

Имаме 3 колони със текст. Да видим какво можем да направим със всяка една от тях.

## Channel Title

In [9]:
channel_titles = videos.channel_title
print(channel_titles.value_counts())

Vox                  40
Jimmy Kimmel Live    40
First We Feast       39
vlogbrothers         38
NFL                  38
jacksfilms           38
TheEllenShow         38
                     ..
Sophia Grace          1
Hot Dad               1
Football Insider      1
ZaynVEVO              1
BadmintonWorld.tv     1
Penny Loggins         1
NikkiPhillippi        1
Name: channel_title, Length: 1230, dtype: int64


Тук ще си позволя да копирам малко код от лекцията :)

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[[self.key]]


In [11]:
from sklearn.preprocessing import LabelBinarizer

class LabelBinarizerPipelineFriendly(LabelBinarizer):
    def fit(self, X, y=None):
        super().fit(X)

    def transform(self, X, y=None):
        return super().transform(X)

    def fit_transform(self, X, y=None):
        return super().fit(X).transform(X)

In [12]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

channel_title_pipeline = Pipeline([
    ('select', ItemSelector('channel_title')),
    ('encoder', LabelBinarizerPipelineFriendly()),
])

channel_title_union = FeatureUnion([
    ('channel', channel_title_pipeline)
])

channel_title_union.fit_transform(videos).shape

(7992, 1230)

Ок, преработихме имената на каналите.. Какво още можем да направим??

## Tags

In [13]:
tags = videos.tags
print(tags.value_counts())

[none]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 491
James Corden|The Late Late Show|Colbert|late night|late night show|Stephen Colbert|Comedy|monologue|comedian|impressions|celebrities|carpool|karaoke|CBS|Late Late Show|Corden|joke|jokes|funny|funny video|funny videos|humor|celebrity|celeb|hollywood|famous                                                                                                                                                                                                                         37
The Late Show|Stephen Colbert|Colbert|Late Show|ce

Тук е малко по-интересно.
Имаме 2204 различни множества от тагове, като някои видеа (на един и същи канал най-вероятно) имат едни и същи тагове.
За сега мисля просто да ги labelBinarize-на (на български) и да видя како ще стане
Други идеи за по-късно евентуално:
- Да сплитна всички по '|' и да енкодна всички (може би ще станат прекалено много фийчъри)
- Горното, но да групирам сходните в едно.

In [14]:
tags_pipeline = Pipeline([
    ('select', ItemSelector('tags')),
    ('encoder', LabelBinarizerPipelineFriendly()),
])

tags_union = FeatureUnion([
    ('tags', tags_pipeline)
])

tags_union.fit_transform(videos).shape

(7992, 2204)

## Title

In [15]:
titles = videos.title
print(titles.value_counts())

Tourist terrified by new glass walkway that cracks under weight      8
IT PENNYWISE GLAM HALLOWEEN MAKEUP TUTORIAL                          8
Kellywise - SNL                                                      8
Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher      8
Hello, world. Meet our baby girl: Alexis Olympia Ohanian, Jr.        7
Harvey Weinstein, Donald Trump and Systemic Sexism: A Closer Look    7
TOMB RAIDER - Official Trailer #1                                    7
                                                                    ..
Spider Face                                                          1
EXTREME SOUR CANDY CHALLENGE *LIVE*                                  1
MACKLEMORE FEAT OTIENO TERRY - LEVITATE                              1
Björk: The Gate                                                      1
Bonnie McKee - Thorns                                                1
SMOKEY MAKEUP LOOK feat CLAUDIA SCHIFFER                             1
DIY TA

Може би не е добра идея да енкодвам имената на видеата по същия начин, но нека го изпробвам

In [16]:
titles_pipeline = Pipeline([
    ('select', ItemSelector('title')),
    ('encoder', LabelBinarizerPipelineFriendly()),
])

titles_union = FeatureUnion([
    ('titles', titles_pipeline)
])

titles_union.fit_transform(videos).shape

(7992, 2398)

# Да претренираме с обработените данни

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

feature_union = FeatureUnion([
    ('channel_title', channel_title_pipeline),
#     ('tags', tags_pipeline), 
#     ('title', titles_pipeline),
    ('likes', Pipeline([
        ('select', ItemSelector('likes')),
        ('scaler', StandardScaler()),
    ])),
    ('dislikes', Pipeline([
        ('select', ItemSelector('dislikes')),
        ('scaler', StandardScaler()),
    ])),
    ('comments', Pipeline([
        ('select', ItemSelector('comment_total')),
        ('scaler', StandardScaler()),
    ]))
])

model = Pipeline([
    ('union', feature_union),
    ('regression', LinearRegression())
])

data = videos[['channel_title', 'tags', 'title', 'likes', 'dislikes', 'comment_total']]

X_train, X_test, y_train, y_test = train_test_split(data, videos.views, random_state=0)

model.fit(X_train, y_train)

print("train score:", model.score(X_train, y_train))
print("test score: ", model.score(X_test, y_test))

train score: 0.933997766117
test score:  -1.08769824296e+21


# Epic Fail
## май овърфитва, не съм сигурен..
### като ползвам и таговете и заглавията става още по-забавно